<a href="https://colab.research.google.com/github/mille-s/ReproHum_072904_DCU25/blob/main/MemSum_Human_Evaluation_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MemSum Human Evaluation

## Clone repos, Install Dependencies, Set working directory (takes about 4 minutes)



In [ ]:
#@title Mount Drive, Set save and load folder (folder that contains the notebooks)
from google.colab import drive
import glob
import os

drive.mount('/content/drive')

gdrive_folder_name = ''
drive_folders = glob.glob('/content/drive/MyDrive/*')
for drive_folder in drive_folders:
  if drive_folder.startswith('/content/drive/MyDrive/2025_ReproHum'):
    head, tail = os.path.split(drive_folder)
    gdrive_folder_name = tail

print(f'Annotation folder: {gdrive_folder_name}')

In [ ]:
#@title Clone MemSum, change working directory
!git clone https://github.com/nianlonggu/MemSum.git

import os
import json
os.chdir("MemSum")

#SM---START---
def prepare_json():
  human_eval_data_orig = [ json.loads(line) for line in open('human_eval_results/records_memsum_wo_autostop_neusum.jsonl',"r") ]
  human_eval_data_orig_no_rankings = []
  x = 0
  while x < len(human_eval_data_orig):
    new_dict = {}
    for key in human_eval_data_orig[x].keys():
      if key != 'ranking_results':
        new_dict[key] = human_eval_data_orig[x][key]
    human_eval_data_orig_no_rankings.append(new_dict)
    x += 1
  with open('human_eval_results/records_memsum_wo_autostop_neusum_clean.jsonl', 'w') as f:
    for item in human_eval_data_orig_no_rankings:
      f.write(json.dumps(item) + '\n')
  !rm /content/MemSum/human_eval_results/records_memsum_wo_autostop_neusum.jsonl
  !rm /content/MemSum/human_eval_results/records_memsum_neusum.jsonl

prepare_json()
#SM---END---

In [ ]:
#@title Requirements
!pip install -r requirements.txt --quiet

In [ ]:
#@title Clone Sent2Vec
!git clone https://github.com/epfml/sent2vec/
!cd sent2vec && git reset --hard 770bd2d && make && pip install .

In [ ]:
#@title Change working directory
os.chdir("../MemSum")

In [ ]:
#@title Load Sent2Vec
import sent2vec

global sent2vec_model
sent2vec_model = sent2vec.Sent2vecModel()
sent2vec_model.load_model('/content/drive/MyDrive/'+gdrive_folder_name+'/wiki_unigrams.bin')

## Utils for evaluation interface

In [ ]:
#@title Eval code
import requests
import json
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, GridspecLayout
from IPython.display import display, Markdown, clear_output
import numpy as np
import pprint
import nltk
from scipy.stats import ttest_rel , ttest_ind, wilcoxon
from scipy.spatial.distance import cosine
import sys
import pickle

#SM---START---
from requests import get
filename = get('http://172.28.0.12:9000/api/sessions').json()[0]['name']
id_notebook = filename.rsplit('_', 1)[1].rsplit('.', 1)[0]

load_partially_completed_file = False #@param{type:"boolean"}
path_file_to_load = "/content/drive/MyDrive/"+gdrive_folder_name+"/saved_annotations1(1)"#+id_notebook
# path_file_to_load = "/content/drive/MyDrive/2025_ReproNLP_Annotators_DCU/saved_annotations"+id_notebook
if load_partially_completed_file == True:
  if path_file_to_load == None:
    print('ERROR: Please enter a filepath.')
    sys.exit()
  else:
    if os.path.exists(path_file_to_load) == False:
      print('ERROR: Filepath does not exist.')
      sys.exit()
    else:
      print('Found file to load.')
#SM---END---

def get_summ_example():
    global human_eval_data, current_doc_idx, num_of_eval_docs

    found = False
    for pos in range( current_doc_idx, len(human_eval_data) ):
        #SM---START---
        # Commented this block, which was probably there for showing the results only (it skips evaluation items)
        # if human_eval_data[pos]["ranking_results"]["overall"] == [1,1]:
        #     human_eval_data[pos]["new_human_eval_results"] = human_eval_data[pos]["ranking_results"]
        #     num_of_eval_docs += 1
        #     if num_of_eval_docs >= len(human_eval_data):
        #         submit_button.disabled = True
        # else:
        #SM---END---
        #SM---START---
        # Unindented this block
        found = True
        current_doc_idx = pos
        break
        #SM---END---
    if found:
        summ_example = human_eval_data[current_doc_idx]
        current_doc_idx = min(current_doc_idx+1, len(human_eval_data) )
    else:
        summ_example = None
        current_doc_idx = len(human_eval_data)

    return summ_example


class TextHTML(widgets.HTML):
    def __init__(self, html_style = {} ,**kwargs):
        super().__init__(**kwargs )
        self.default_html_style = {
            "padding":"5px",
            "height":"600px",
            "overflow-x":"hidden",
            "border":"1px solid grey",
            "line-height":"20px"
         }
        self.render_sen_list(html_style=html_style)
        self.html_lines = []

    def render_sen_list(self, sens=[], html_style = {}):
        self.default_html_style.update(html_style)
        html_lines = [
            '''<div style="%s">''' %( "; ".join( ":".join([key, value]) for key, value in self.default_html_style.items() )  )
        ]

        for sen in sens:
            is_marked = sen.get("is_marked", False)
            sen_text = sen.get("text", "").capitalize()
            html_line = '''<p> %s %s %s</p>'''%( '''<span style="background-color: #FFFF00">''' if is_marked else "",
                                             sen_text,
                                             '''</span>''' if is_marked else ""
                                           )
            html_lines.append( html_line )

        html_lines.append( "</div>" )
        value = "\n".join(html_lines)
        self.value = value
        self.html_lines = html_lines

    def update_html_style( self, html_style = {} ):
        self.default_html_style.update(html_style)
        self.html_lines[0] = '''<div style="%s">''' %( "; ".join( ":".join([key, value]) for key, value in self.default_html_style.items() )  )
        value = "\n".join(self.html_lines)
        self.value = value


def get_cosine_sim(query, sentences, model):
  query_vec = model.embed_sentence(query)
  sentences_vec = model.embed_sentences(sentences)
  cosine_sim = [ 1-cosine(query_vec[0], sent_vec) for sent_vec in sentences_vec ]
  return cosine_sim


# Function to highlight text when the button is pressed
def on_highlight_button_click(change):
    # Update the HTML for each summary with highlighted text
    global summ_example, query_text, sent2vec_model
    query = query_text.value

    # highlight query

    summaryA = summ_example["random_extracted_results"][0][0]
    summaryA_cosine_sim = get_cosine_sim(query, summaryA, sent2vec_model)
    list_sent = []
    for idx, sent in enumerate(summaryA):
      if summaryA_cosine_sim[idx]>0.6:
        list_sent.append({"text":f'<mark>{sent}</mark>'})
      else:
        list_sent.append({"text":sent})
    text_summ_sources["Summary A"].render_sen_list( list_sent )

    summaryB = summ_example["random_extracted_results"][1][0]
    summaryB_cosine_sim = get_cosine_sim(query, summaryB, sent2vec_model)
    list_sent = []
    for idx, sent in enumerate(summaryB):
      if summaryB_cosine_sim[idx]>0.6:
        list_sent.append({"text":f'<mark>{sent}</mark>'})
      else:
        list_sent.append({"text":sent})
    text_summ_sources["Summary B"].render_sen_list( list_sent )


form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

rb_criteria = {}
for criterion in ["Overall:"]:
    rb_criteria[criterion] =  widgets.RadioButtons(
                options=['summary A', 'summary B'],
                disabled=False,
                index = None
    )

b_summ_sources = {}
colors_for_b_summ_sources ={ "Reference Summary":"YellowGreen","Summary A":"lightblue", "Summary B":"lightblue" }
for source in ["Reference Summary", "Summary A", "Summary B"]:
    b_summ_sources[source] = Button(description=source, layout=Layout(height='auto', width='auto'))
    b_summ_sources[source].style.button_color = colors_for_b_summ_sources[source]

text_summ_sources = {}
for source in ["Reference Summary", "Summary A", "Summary B"]:
    text_summ_sources[source] =  TextHTML({"height":"500px"})  # Textarea(layout=Layout(height="600px", width='auto'))

submit_button = Button(description="Submit & Eval Next", layout=Layout(height='auto', width='auto'))
submit_button.style.button_color = "LightSalmon"


global fulltext_textbox
fulltext_button = Button(description="Show Source Document >>>", layout=Layout(height='auto', width='32.9%'))
fulltext_textbox = TextHTML( html_style={"height":"0px"}, layout=Layout(visibility="hidden") )

grid_b_summ = GridspecLayout(1,3)
grid_b_summ[0,0] = b_summ_sources["Reference Summary"]
grid_b_summ[0,1] = b_summ_sources["Summary A"]
grid_b_summ[0,2] = b_summ_sources["Summary B"]
grid_text_summ = GridspecLayout(1,3)
grid_text_summ[0,0] = text_summ_sources["Reference Summary"]
grid_text_summ[0,1] = text_summ_sources["Summary A"]
grid_text_summ[0,2] = text_summ_sources["Summary B"]
grid_rb_description = GridspecLayout(1,3)
grid_rb_description[0,0] = Label(value = "Overall:")


global query_text
highlight_button = Button(description="Highlight relevant sentences given a query 🔍", layout=Layout(height='auto', width='auto'))
highlight_button.on_click(on_highlight_button_click)
query_text = Textarea(placeholder='Enter your query here...', layout=Layout(width='auto', height='auto'))

grid_query = GridspecLayout(1,3)
grid_query[0,0] = highlight_button
grid_query[0,1:] = query_text

output_panel = widgets.Output()


form_items = [
    widgets.HTML(value = f"<b><font color='black' font size='4pt'>Read</b>"),
    grid_query,
    grid_b_summ,
    grid_text_summ,
    fulltext_button,
    fulltext_textbox,
    widgets.HTML(value = f"<b><font color='black' font size='4pt'>Evaluation (choose one that is closer to the reference summary)</b>"),
    grid_rb_description,
    widgets.HBox([ rb_criteria["Overall:"] ], layout=form_item_layout),
    widgets.Box([  submit_button ], layout=form_item_layout),
    output_panel

]

gui = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='100%'
))


def get_next_example():
    global summ_example, num_of_eval_docs, human_eval_data
    summ_example = get_summ_example()
    if summ_example is not None:
        text_summ_sources["Reference Summary"].render_sen_list( [{"text":_} for _ in summ_example["summary"]] )
        text_summ_sources["Summary A"].render_sen_list( [{"text":_} for _ in summ_example["random_extracted_results"][0][0] ] )
        text_summ_sources["Summary B"].render_sen_list( [{"text":_} for _ in summ_example["random_extracted_results"][1][0] ] )
    else:
        text_summ_sources["Reference Summary"].render_sen_list( [] )
        text_summ_sources["Summary A"].render_sen_list( [] )
        text_summ_sources["Summary B"].render_sen_list( [] )


    for criterion in ["Overall:"]:
        rb_criteria[criterion].index = None

    if summ_example is not None:
        fulltext_textbox.render_sen_list( [{"text":_} for _ in summ_example["text"]] )
    else:
        fulltext_textbox.render_sen_list( [] )

    fulltext_textbox.update_html_style({"height":"0px"})
    fulltext_textbox.layout.visibility = "hidden"
    fulltext_button.description = "Show Source Document >>>"

    # empty search textbox
    query_text.value = ""

def fulltext_button_on_click_listener(_):
    if fulltext_button.description == "Show Source Document >>>":
        fulltext_textbox.update_html_style({"height":"600px"})
        fulltext_textbox.layout.visibility = "visible"
        fulltext_button.description = "Hide Source Document >>>"
    elif fulltext_button.description == "Hide Source Document >>>":
        fulltext_textbox.update_html_style({"height":"0px"})
        fulltext_textbox.layout.visibility = "hidden"
        fulltext_button.description = "Show Source Document >>>"
fulltext_button.on_click( fulltext_button_on_click_listener )


def submit_button_on_click_listener(_):
    global summ_example, num_of_eval_docs
    all_evaluated = True
    for criterion in ["Overall:"]:
        if rb_criteria[criterion].index is None:
            with output_panel:
                clear_output()
                print("You have not evaluated %s, please retry."%( criterion.rstrip(":") ))
            all_evaluated = False
    if all_evaluated:
        two_orders = [ [1,2],[2,1] ]
        summ_example["new_human_eval_results"] = {
                                         "overall":two_orders[ rb_criteria["Overall:"].index ]
                                    }
        num_of_eval_docs += 1
        if num_of_eval_docs >= len(human_eval_data):
            submit_button.disabled = True
        else:
            get_next_example()
        with output_panel:
            clear_output()
            print("You have evaluated %d/%d examples."%( num_of_eval_docs, len(human_eval_data)))

    #SM---START---
    # print(num_of_eval_docs, current_doc_idx)
    # print(human_eval_data[num_of_eval_docs-1]["new_human_eval_results"])
    with open(path_file_to_load, 'wb') as f:
        pickle.dump(human_eval_data, f)
    #SM---END---

submit_button.on_click( submit_button_on_click_listener )

human_eval_data = None
summ_example = None
num_of_eval_docs = 0
current_doc_idx = 0

def run_gui( dataset_path,  width ="90%", textbox_height = "400px", ):
    global human_eval_data, summ_example, num_of_eval_docs, current_doc_idx
    #SM---START---
    num_annnotations_loaded = 0
    if load_partially_completed_file == True:
        if load_partially_completed_file == True:
            with open(path_file_to_load, 'rb') as f:
                human_eval_data = pickle.load(f)
                for z in range(len(human_eval_data)):
                    if 'new_human_eval_results' in human_eval_data[z].keys():
                        num_annnotations_loaded += 1
    else:
    #SM---END---
        human_eval_data = [ json.loads(line) for line in open(dataset_path,"r") ]

    summ_example = None
    num_of_eval_docs = 0
    current_doc_idx = 0
    #SM---START---
    if load_partially_completed_file == True:
        num_of_eval_docs = num_annnotations_loaded
        current_doc_idx = num_annnotations_loaded
    #SM---END---
    get_next_example()
    with output_panel:
        clear_output()
        print("You have evaluated %d/%d examples."%( num_of_eval_docs, len(human_eval_data)))

    for source in ["Reference Summary", "Summary A", "Summary B"]:
        text_summ_sources[source].update_html_style({"height":textbox_height})
    gui.layout.width = width
    return gui

## Human Evaluation Experiment II:

In [ ]:
run_gui(dataset_path = "human_eval_results/records_memsum_wo_autostop_neusum_clean.jsonl")

## Debug (for team carrying out the reproduction)

In [ ]:
human_eval_data[0]

In [ ]:
#@title Control loaded file
with open(path_file_to_load, 'rb') as f:
  human_eval_data_load = pickle.load(f)
  for z in range(len(human_eval_data_load)):
    print(z)
    if 'new_human_eval_results' in human_eval_data_load[z].keys():
      print(human_eval_data_load[z]['new_human_eval_results'])

In [ ]:
#@title Control human_eval_data list: new annotations
print(len(human_eval_data), 'datapoints.')
print(human_eval_data[0].keys())
print()

x = 0
while x < num_of_eval_docs:
  print(human_eval_data[x]['new_human_eval_results'])
  x += 1

In [ ]:
#@title Control human_eval_data list: existing data
human_eval_data_orig = [ json.loads(line) for line in open('human_eval_results/records_memsum_wo_autostop_neusum_clean.jsonl',"r") ]

print(len(human_eval_data_orig), 'datapoints.')
print(human_eval_data_orig[0].keys())
print()

x = 0
while x < len(human_eval_data_orig):
  print(str(x), human_eval_data_orig[x]['random_orders'], human_eval_data_orig[x]['unique_id'])
  # print(human_eval_data_orig[x].keys())
  x += 1

In [ ]:
#@title Print examples
print(summ_example['indice'])
print(summ_example['summary'])
print('----------------------')
summ_example['random_extracted_results']

## Process results (for team carrying out the reproduction)

In [ ]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Set directory
ReproNLP_dir = '/content/drive/MyDrive/2025_ReproNLP_Annotators_DCU/evals_results'

In [ ]:
#@title Load annotated files and get averages
import pandas as pd
import glob
import os
import json
import pickle

folders_results  = sorted(glob.glob(ReproNLP_dir + '/*'))

def scores2df(path_file_to_load, count_items, df_scores, newfile = True):
  first_item_id = count_items
  if newfile == True:
    # In case of "regular" files (i.e. one file containing all items), start numbering from 0. In case of items distributed in 2 files, keep counting from the end of the previous file
    first_item_id = 0
  with open(path_file_to_load, 'rb') as f:
    # Load file with scores
    human_eval_data_load = None
    annotator_id = None
    # That's the key of the dico hwhere the results can be found (different names in original and new files)
    rankings_key = None
    if '.jsonl' in path_file_to_load:
      # That's for loading the original experiment json file
      human_eval_data_load = [ json.loads(line) for line in open(path_file_to_load,"r") ]
      rankings_key = 'ranking_results'
      # print(human_eval_data_load)
    else:
      # That's for loading our new evaluation files collected during the reproduction
      # Get annotator ID from path
      annotator_id = os.path.split(path_file_to_load)[1].split('saved_annotations')[1][0]
      human_eval_data_load = pickle.load(f)
      rankings_key = 'new_human_eval_results'
    # Get scores from the first item to the last (annotator 1 has their scores in 2 files in the reproduction, the first item if the second file is item 35)
    for z in range(first_item_id, len(human_eval_data_load)):
      # print(human_eval_data_load[z].keys())
      if rankings_key in human_eval_data_load[z].keys():
        # print(count_items)
        # Get rankings of systems (list [1, 2] or [2, 1])
        ranks = human_eval_data_load[z][rankings_key]['overall']
        assert len(ranks) == 2, f'There should be 2 values in the ranks list, found {len(ranks)}.'
        # The next one is deactivated because in the original data there are ties [1, 1]
        # assert ranks[0] != ranks[1], f'There should be 2 different values in the ranks list, found {ranks[0]} and {ranks[1]}.'
        assert ranks[0] == 1 or ranks[0] == 2, f'The first value in the ranks list should be 1 or 2, found {ranks[0]}.'
        assert ranks[1] == 1 or ranks[1] == 2, f'The second value in the ranks list should be 1 or 2, found {ranks[1]}.'
        # Get the system order (['MemSum', 'NeuSum'] or ['NeuSum', 'MemSum'])
        systems = human_eval_data_load[z]['random_orders']
        # Get the position of each system in the system order list
        memsum_pos = systems.index('MemSum')
        neusum_pos = systems.index('NeuSum')
        # print(ranks, systems)
        # print(f'memsum: {ranks[memsum_pos]}')
        # print(f'neusum: {ranks[neusum_pos]}')
        # Get the rank of each system by matching the position in the system list and the position in the rank list
        df_scores.loc[count_items] = [annotator_id, z, ranks[memsum_pos], ranks[neusum_pos]]
        count_items += 1
  return(df_scores, count_items)

def get_mean_scores(df_scores, count_files):
  # Average scores per annotator
  memsum_avg_by_annot = round(df_scores.groupby(['id_annot']).agg(MemSum=('MemSum', 'mean')), 2)
  neusum_avg_by_annot = round(df_scores.groupby(['id_annot']).agg(NeuSum=('NeuSum', 'mean')), 2)
  # Average scores across annotators
  memsum_avg = round(df_scores['MemSum'].mean(), 2)
  neusum_avg = round(df_scores['NeuSum'].mean(), 2)
  print()
  print(memsum_avg_by_annot)
  print()
  print(neusum_avg_by_annot)
  print()
  print(f'MemSum average ({count_files} evaluators): {memsum_avg}')
  print(f'NeuSum average ({count_files} evaluators): {neusum_avg}')
  return memsum_avg, neusum_avg

def aggregate_scores_per_item(df_scores):
  # print(df_scores.groupby(['id_item']).agg(MemSum=('MemSum', 'mean')))
  # print(df_scores.groupby(['id_item']).agg(NeuSum=('NeuSum', 'mean')))
  df_scores_per_item = (df_scores.groupby(['id_item']).sum())
  for index, row in df_scores_per_item.iterrows():
    if row['MemSum'] < row['NeuSum']:
      # Add a column to store the aggregated score for each system (1 or 2)
      df_scores_per_item.loc[index, 'MemSum'] = 1
      df_scores_per_item.loc[index, 'NeuSum'] = 2
    elif row['MemSum'] > row['NeuSum']:
      # Add a column to store the aggregated score for each system (1 or 2)
      df_scores_per_item.loc[index, 'MemSum'] = 2
      df_scores_per_item.loc[index, 'NeuSum'] = 1
    else:
      # Add a column to store the aggregated score for each system (1 or 2)
      df_scores_per_item.loc[index, 'MemSum'] = 1
      df_scores_per_item.loc[index, 'NeuSum'] = 1
  display(df_scores_per_item)
  return df_scores_per_item

# Make dataframe headers
df_scores = pd.DataFrame(columns=['id_annot', 'id_item', 'MemSum', 'NeuSum'])
count_items = 0
count_files = 0
for folder_results in folders_results:
  id_notebook = os.path.basename(folder_results)
  path_file_to_load = os.path.join(folder_results, 'saved_annotations'+str(id_notebook))
  if os.path.exists(path_file_to_load):
    print(f'Processing {path_file_to_load}...')
    # If we have a fully annotated file (original annnotators 2, 3 and 4), simply use the file
    df_scores, count_items = scores2df(path_file_to_load, count_items, df_scores)
    print(f'  {count_items} total rankings collected...')
    count_files += 1
  else:
    # For one annotator, we have to combine scores in 2 files
    if 'saved_annotations1' in path_file_to_load:
      path_file_to_load1 = os.path.join(ReproNLP_dir, '1', 'saved_annotations1_first35')
      path_file_to_load2 = os.path.join(ReproNLP_dir, '1', 'saved_annotations1_last28')
      print(f'Processing /content/drive/MyDrive/2025_ReproNLP_Annotators_DCU/evals_results/2/saved_annotations1_1-35...')
      df_scores, count_items = scores2df(path_file_to_load1, count_items, df_scores)
      print(f'  {count_items} total rankings collected...')
      print(f'Processing /content/drive/MyDrive/2025_ReproNLP_Annotators_DCU/evals_results/2/saved_annotations1_36-63...')
      # newfile == True (True is the default value in the function args) means that the first item counter is reinitialised (so the 63 rows for each annotator are numbered from 0 to 62)
      df_scores, count_items = scores2df(path_file_to_load2, count_items, df_scores, newfile = False)
      print(f'  {count_items} total rankings collected...')
      count_files += 1

assert count_items/count_files == 63, f'Error, there should be 63 items per annotator, fount {count_items/count_files}.'
print('63 ratings per annotator were found.')
print()
print(df_scores)

# Compute and print average scores over all 252 ratings
memsum_avg_repro, neusum_avg_repro = get_mean_scores(df_scores, count_files)

# Aggregate scores per evalaution item (looks like that's what was done in the original paper)
df_scores_per_item = aggregate_scores_per_item(df_scores)
# Compute and print average scores over all 63 aggregated ratings
memsum_avg_repro_agg, neusum_avg_repro_agg = get_mean_scores(df_scores_per_item, count_files)

In [ ]:
#@title Process the original results file and get averages

assert not os.path.exists('/content/MemSum/human_eval_results/records_memsum_wo_autostop_neusum_clean.jsonl'), 'You need to delete and restart the runtime NOT RUN the "Clone MemSum, change working directory" cell above!'
if not os.path.exists('/content/MemSum'):
  !git clone https://github.com/nianlonggu/MemSum.git

dataset_path = '/content/MemSum/human_eval_results/records_memsum_wo_autostop_neusum.jsonl'

df_scores_new = pd.DataFrame(columns=['id_annot', 'id_item', 'MemSum', 'NeuSum'])
count_items = 0
df_scores_orig, count_items = scores2df(dataset_path, count_items, df_scores_new)
count_files = 1
print(f'  {count_items} total rankings collected...')

memsum_avg_orig, neusum_avg_orig = get_mean_scores(df_scores_orig, count_files)
print()
print('Results in paper: MemSum=1.38, NeuSum=1.57')

In [ ]:
#@title Make CSV file for running QRA+ code

# Make CSV for scores not averaged
df_csv_qra = pd.DataFrame(columns=['Key', 'Paper', 'Study', 'System', 'Criterion', 'Result'])
df_csv_qra.loc[0] = ['0729-04', 'Gu et al 2022', 'Original', 'MemSum', 'Overall', memsum_avg_orig]
df_csv_qra.loc[1] = ['0729-04', 'Gu et al 2022', 'Original', 'NeuSum', 'Overall', neusum_avg_orig]
df_csv_qra.loc[2] = ['0729-04', 'Gu et al 2022', 'Reproduction 1', 'MemSum', 'Overall', memsum_avg_repro]
df_csv_qra.loc[3] = ['0729-04', 'Gu et al 2022', 'Reproduction 1', 'NeuSum', 'Overall', neusum_avg_repro]

# Save df_csv_qra in csv file
df_csv_qra.to_csv('/content/gu-etal-2022_DCU.csv', index=False)
print('Created /content/gu-etal-2022_DCU.csv')

# Make CSV for item-level averaged scores
df_csv_qra_agg = pd.DataFrame(columns=['Key', 'Paper', 'Study', 'System', 'Criterion', 'Result'])
df_csv_qra_agg.loc[0] = ['0729-04', 'Gu et al 2022', 'Original', 'MemSum', 'Overall', memsum_avg_orig]
df_csv_qra_agg.loc[1] = ['0729-04', 'Gu et al 2022', 'Original', 'NeuSum', 'Overall', neusum_avg_orig]
df_csv_qra_agg.loc[2] = ['0729-04', 'Gu et al 2022', 'Reproduction 1', 'MemSum', 'Overall', memsum_avg_repro_agg]
df_csv_qra_agg.loc[3] = ['0729-04', 'Gu et al 2022', 'Reproduction 1', 'NeuSum', 'Overall', neusum_avg_repro_agg]

# Save df_csv_qra in csv file
df_csv_qra_agg.to_csv('/content/gu-etal-2022_DCU_agg.csv', index=False)
print('Created /content/gu-etal-2022_DCU_agg.csv')

In [ ]:
#@title Calculate Fleiss's Kappa

'''
Created on Aug 1, 2016
@author: skarumbaiah

Computes Fleiss' Kappa
Joseph L. Fleiss, Measuring Nominal Scale Agreement Among Many Raters, 1971.
'''

def checkInput(rate, n):
    """
    Check correctness of the input matrix
    @param rate - ratings matrix
    @return n - number of raters
    @throws AssertionError
    """
    N = len(rate)
    k = len(rate[0])
    assert all(len(rate[i]) == k for i in range(k)), "Row length != #categories)"
    assert all(isinstance(rate[i][j], int) for i in range(N) for j in range(k)), "Element not integer"
    assert all(sum(row) == n for row in rate), "Sum of ratings != #raters)"

def fleissKappa(rate,n):
    """
    Computes the Kappa value
    @param rate - ratings matrix containing number of ratings for each subject per category
    [size - N X k where N = #subjects and k = #categories]
    @param n - number of raters
    @return fleiss' kappa
    """

    N = len(rate)
    k = len(rate[0])
    print("#raters = ", n, ", #subjects = ", N, ", #categories = ", k)
    checkInput(rate, n)

    #mean of the extent to which raters agree for the ith subject
    PA = sum([(sum([i**2 for i in row])- n) / (n * (n - 1)) for row in rate])/N
    print("PA = ", PA)

    # mean of squares of proportion of all assignments which were to jth category
    PE = sum([j**2 for j in [sum([rows[i] for rows in rate])/(N*n) for i in range(k)]])
    print("PE =", PE)

    kappa = -float("inf")
    try:
        kappa = (PA - PE) / (1 - PE)
        kappa = float("{:.3f}".format(kappa))
    except ZeroDivisionError:
        print("Expected agreement = 1")

    print("Fleiss' Kappa =", kappa)

    return kappa

# Code above expects a matrix of d rows and s columns where:
# d is the number of data points (also called "subjects")
# s is the number of categories, i.e. possible scores (in our case, for one system, that's 2: 1 and 2).
# Then at each position in the matrix, there should be the number n of annotators who chose the score s for datapoint d.
# rate = [
#     [2,0,0,0],
#     [0,2,0,0],
#     [0,0,2,0],
#     [0,0,0,2]
# ]
# kappa = fleissKappa(rate,2)
# assert(kappa==1)

def makeFleissMatrix(df_scores):
  # get list with scores of each annotator for each input_id in df
  scores_matrix = []
  # Group rows per annotation item
  df_scores_per_item = df_scores.groupby(['id_item'])
  for key, item in df_scores_per_item:
    # print(key[0])
    count_memsum_first = 0
    count_neusum_first = 0
    # print(df_scores_per_item.get_group(key))
    # Get the MemSum score for each annotator
    for row in df_scores_per_item.get_group(key).iterrows():
      if row[1]['MemSum'] == 1:
        count_memsum_first += 1
      elif row[1]['NeuSum'] == 1:
        count_neusum_first += 1
    assert count_memsum_first + count_neusum_first == 4, f'There should be 4 first ranks, found {count_memsum_first + count_neusum_first}.'
    # print(f'  MemSum 1st/2nd count: {count_memsum_first}-{count_neusum_first}\n')
    # Put counts in the matrix
    scores_matrix.append([count_memsum_first, count_neusum_first])
  return scores_matrix

# Prepare matrix for fleiss
scores_matrix = makeFleissMatrix(df_scores)
print(scores_matrix)

# Calculate fleiss kappa
kappa = fleissKappa(scores_matrix,4)

# scores_matrix_test1 = [[4, 0], [3, 1], [0, 4], [4, 0], [1, 3], [4, 0], [0, 4], [3, 1], [3, 1], [1, 3], [4, 0], [0, 4], [1, 3], [3, 1], [3, 1], [3, 1], [4, 0], [4, 0], [3, 1], [1, 3], [1, 3], [4, 0], [3, 1], [4, 0], [4, 0], [0, 4], [0, 4], [4, 0], [4, 0], [4, 0], [4, 0], [4, 0], [3, 1], [1, 3], [1, 3], [3, 1], [0, 4], [4, 0], [0, 4], [3, 1], [1, 3], [0, 4], [1, 3], [0, 4], [3, 1], [0, 4], [1, 3], [0, 4], [3, 1], [4, 0], [4, 0], [1, 3], [4, 0], [0, 4], [0, 4], [1, 3], [1, 3], [0, 4], [0, 4], [1, 3], [3, 1], [3, 1], [0, 4]]
# kappa = fleissKappa(scores_matrix_test1,4)
# scores_matrix_test2 = [[4, 0], [4, 0], [0, 4], [4, 0], [0, 4], [4, 0], [0, 4], [4, 0], [4, 0], [0, 4], [4, 0], [0, 4], [0, 4], [4, 0], [4, 0], [4, 0], [4, 0], [4, 0], [4, 0], [0, 4], [0, 4], [4, 0], [4, 0], [4, 0], [4, 0], [0, 4], [0, 4], [4, 0], [4, 0], [4, 0], [4, 0], [4, 0], [4, 0], [0, 4], [0, 4], [4, 0], [0, 4], [4, 0], [0, 4], [4, 0], [0, 4], [0, 4], [0, 4], [0, 4], [4, 0], [0, 4], [0, 4], [0, 4], [4, 0], [4, 0], [4, 0], [0, 4], [4, 0], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [4, 0], [4, 0], [0, 4]]
# kappa = fleissKappa(scores_matrix_test2,4)

In [ ]:
#@title Calculate Wilcoxon signed-rank test
import scipy.stats as stats

data_to_use = 'original_agg'#@param['original_agg', 'repro_all', 'repro_agg']

scores_memsum_wilcox = None
scores_neusum_wilcox = None
if data_to_use == 'original_agg':
  print('Processing original aggregated results file...')
  scores_memsum_orig = df_scores_orig['MemSum'].tolist()
  scores_neusum_orig = df_scores_orig['NeuSum'].tolist()
  scores_memsum_wilcox = scores_memsum_orig
  scores_neusum_wilcox = scores_neusum_orig
elif data_to_use == 'repro_all':
  print('Processing reproduction results file...')
  scores_memsum_repro = df_scores['MemSum'].tolist()
  scores_neusum_repro = df_scores['NeuSum'].tolist()
  scores_memsum_wilcox = scores_memsum_repro
  scores_neusum_wilcox = scores_neusum_repro
elif data_to_use == 'repro_agg':
  print('Processing reproduction aggregated results file...')
  scores_memsum_repro_agg = df_scores_per_item['MemSum'].tolist()
  scores_neusum_repro_agg = df_scores_per_item['NeuSum'].tolist()
  scores_memsum_wilcox = scores_memsum_repro_agg
  scores_neusum_wilcox = scores_neusum_repro_agg

stats.wilcoxon(scores_memsum_wilcox, scores_neusum_wilcox)